<a href="https://colab.research.google.com/github/naldo14/tesis-/blob/main/casewest_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import h5py #librerua para crear archivos , h5py
import sys
import numpy as np
import math
from scipy import signal    #aqui estoy usando la libreria signal
import scipy.io
import os
import pandas as pd
import pywt
import tensorflow_probability as tfp
from tensorflow import keras
tfd = tfp.distributions
tfpl = tfp.layers
from tensorflow.keras.callbacks import LearningRateScheduler

In [2]:
#hacer el mounted de mi cuenta drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#celda para crear la lista con los paths de mi data
paths=[] #creo la lista de pahts
names=[] #creo la lista de nombres
output_file=[]
paths.append('/content/drive/MyDrive/tesis USB /case west reserve university')
names.append('casewest')
output_file.append('/content/drive/MyDrive/tesis USB /case west reserve university/data_1')

In [3]:
#hago el unar del archivo
!pip install rarfile

from google.colab import drive
import rarfile

# Path to the .zip file in Google Drive
zip_file_path ="/content/drive/MyDrive/tesis USB /case west reserve university/data_1/data.rar"

with rarfile.RarFile(zip_file_path, 'r') as rf:
    rf.extractall(output_file[0])

NameError: ignored

In [ ]:
# #creo que los h5py file vacios
# import os
# import h5py
# # List of directory paths
# directory_paths = [
#     "/content/drive/MyDrive/tesis USB /case west reserve university/data_1/inner",
#     "/content/drive/MyDrive/tesis USB /case west reserve university/data_1/outter",
#     "/content/drive/MyDrive/tesis USB /case west reserve university/data_1/outter1",
#     "/content/drive/MyDrive/tesis USB /case west reserve university/data_1/normal",
#     "/content/drive/MyDrive/tesis USB /case west reserve university/data_1/ball"
# ]

# # Loop through each directory path
# for directory_path in directory_paths:
#     # Get the directory name (e.g., "inner," "outter," etc.)
#     directory_name = os.path.basename(directory_path)

#     # Create an h5py file for the directory
#     h5py_file = h5py.File(f"{directory_name}.h5", "w")

#     # List .mat files in the directory
#     mat_files = [f for f in os.listdir(directory_path) if f.endswith(".mat")]

#     # Add each .mat file to the h5py file as an empty dataset
#     for mat_file in mat_files:
#         # Create an empty dataset
#         dataset = h5py_file.create_dataset(mat_file, shape=(0,), dtype='f4')

#     # Close the h5py file
#     h5py_file.close()

file_names = ["inner.h5", "outter.h5", "normal.h5","outter1.h5","ball.h5"]
drive_path="/content/drive/MyDrive/tesis USB /case west reserve university/data_1"
for file_name in file_names:
    h5_file_path = os.path.join(drive_path, file_name)
    with h5py.File(h5_file_path, 'w') as h5file:
        # You can optionally add attributes, datasets, or groups to the file
        # h5file.create_dataset("dataset_name", data=None)
        # h5file.create_group("group_name")
        # h5file.attrs["attribute_name"] = "attribute_value"
        print("Empty H5PY files created in Google Drive.")


Empty H5PY files created in Google Drive.
Empty H5PY files created in Google Drive.
Empty H5PY files created in Google Drive.
Empty H5PY files created in Google Drive.
Empty H5PY files created in Google Drive.


In [ ]:

### function to storaged all the .mat file into the h5py's file
def save_mat_to_h5py(mat_files_directories, h5py_file_names):
    if len(mat_files_directories) != len(h5py_file_names):
        print("Error: The number of directories and h5py file names should match.")
        return

    for mat_files_directory, h5py_file_name in zip(mat_files_directories, h5py_file_names):
        # Open the h5py file
        with h5py.File(h5py_file_name, 'w') as h5file:
            # Get the list of .mat files in the directory
            mat_files = [f for f in os.listdir(mat_files_directory) if f.endswith(".mat")]

            # Add each .mat file content to the h5py file
            for mat_file in mat_files:
                # Load the ..mat file
                mat_data = scipy.io.loadmat(os.path.join(mat_files_directory, mat_file))

                # Create datasets for each variable in the .mat file
                for var_name, var_data in mat_data.items():
                    h5file.create_dataset(f"{mat_file}/{var_name}", data=var_data)

mat_files_directories = [
    '/content/drive/MyDrive/tesis USB /case west reserve university/data_1/data/inner',
    '/content/drive/MyDrive/tesis USB /case west reserve university/data_1/data/normal',
    '/content/drive/MyDrive/tesis USB /case west reserve university/data_1/data/outter',
    '/content/drive/MyDrive/tesis USB /case west reserve university/data_1/data/outter1',
    '/content/drive/MyDrive/tesis USB /case west reserve university/data_1/data/ball'

]
h5py_file_names = [
    '/content/drive/MyDrive/tesis USB /case west reserve university/data_1/inner.h5',
    '/content/drive/MyDrive/tesis USB /case west reserve university/data_1/normal.h5',
    '/content/drive/MyDrive/tesis USB /case west reserve university/data_1/outter.h5',
    '/content/drive/MyDrive/tesis USB /case west reserve university/data_1/outter1.h5',
    '/content/drive/MyDrive/tesis USB /case west reserve university/data_1/ball.h5'
]
save_mat_to_h5py(mat_files_directories, h5py_file_names)

In [16]:
# 'data' now contains a dictionary where the keys are the .mat file names
# and the values are dictionaries containing the arrays for each .mat file

def get_h5py_data(h5py_file_path):
    data = {}

    # Open the h5py file
    with h5py.File(h5py_file_path, 'r') as h5file:
        # List the keys (dataset names) in the h5py file
        keys = list(h5file.keys())

        for key in keys:
            # Access the content of the .mat file (dataset)
            mat_data = h5file[key]

            # Create a dictionary to store the arrays for this .mat file
            arrays = {}

            for subkey, value in mat_data.items():
                if isinstance(value, h5py.Group):
                    # If it's a group (nested data), you can access its keys
                    subkeys = list(value.keys())
                    for subkey in subkeys:
                        # Access the actual data (arrays) and store in the dictionary
                        subkey_data = value[subkey][()]
                        arrays[subkey] = subkey_data
                else:
                    # If it's a dataset (e.g., a variable), you can access its value
                    # and store it in the dictionary using the original key
                    variable_data = value[()]
                    arrays[subkey] = variable_data

            # Store the arrays for this .mat file in the data dictionary
            data[key] = arrays

    return data


In [4]:

def band_maker(coeffs, fs, title):
    p = len(coeffs)- 1
    low_frequencies = [None] * len(coeffs)
    high_frequencies = [None] * len(coeffs)

    # low_frequencies[0] = 0
    # high_frequencies[0] = fs / (2 ** (p) * 2)
    for i in range(0, len(coeffs)):
         high_frequencies[i] = fs / (2 ** (p+-i + 1))
    low_frequencies[0]=0
    low_frequencies[1:len(low_frequencies)]= high_frequencies[:len(low_frequencies) - 1]



    return  high_frequencies,low_frequencies

In [5]:
def frequencies_maker(N,fs,d,D,theta):
  theta=(theta/180)*np.pi
  FTF=(fs/2)*(1-d/D*np.cos(theta))
  BPFI=(N*fs/2)*(1+d/D*np.cos(theta))
  BPFO=(N*fs/2)*(1-d/D*np.cos(theta))
  BSF=(D*fs/d)*(1-(d/D)**2*np.cos(theta))
  return(FTF,BPFI,BPFO,BSF)


In [11]:
FTF,BPFI,BPFO,BSF=frequencies_maker(9,50.17,7.90,38.5,0) #estos datos son para el daata set coreano
data = {
    'Name': ['Fundamental train frequency (FTF)', 'Ball pass frequency inner (BPFI)', 'Ball pass frequency outer (BPFO)','Ball spin frequency (BSF)'],
    'Values (HZ)': [FTF, BPFI, BPFO,BSF]
}

# Create a dataframe from the dictionary
df = pd.DataFrame(data)

# Print the dataframe
print(df)

                                Name  Values (HZ)
0  Fundamental train frequency (FTF)    19.937688
1   Ball pass frequency inner (BPFI)   272.090805
2   Ball pass frequency outer (BPFO)   179.439195
3          Ball spin frequency (BSF)   234.204744


In [7]:
def cwt_maker(dic_data, wavelet='db38', level=5):
  w = pywt.Wavelet(wavelet)
  coeffs = pywt.wavedec(dic_data, w, level=level)
  print(level)
  energies_levels = [np.sum(np.abs(coeffs)**2) for coeffs in coeffs]
  array_dwt = energies_levels

  return (np.array(array_dwt))


In [8]:
#aqui solo normalizo los niveles energeticos

def normalization_from_coeffs(coeffs):

    mean_energy = np.mean(coeffs)
    std_energy = np.std(coeffs)

    # Calculate Z-scores
    z_scores = [(energy - mean_energy) / std_energy for energy in coeffs]

    return z_scores

In [ ]:
## almacenamiento de la data inner
file_path = '/content/drive/MyDrive/tesis USB /case west reserve university/data_1/inner.h5'
data_inner = get_h5py_data(file_path)

In [ ]:
data_inner.keys()

dict_keys(['109.mat', '110.mat', '111.mat', '112.mat', '174.mat', '175.mat', '176.mat', '177.mat', '213.mat', '214.mat', '215.mat', '217.mat'])

In [ ]:
## almacenamiento de la data outter
file_path = '/content/drive/MyDrive/tesis USB /case west reserve university/data_1/outter.h5'
data_outter = get_h5py_data(file_path)

In [ ]:
data_outter.keys()

dict_keys(['135.mat', '136.mat', '137.mat', '138.mat', '190.mat', '191.mat', '192.mat', '201.mat', '202.mat', '204.mat', '226.mat', '227.mat', '228.mat', '238.mat', '239.mat', '240.mat', '241.mat'])

In [ ]:
## almacenamiento de la data outter
file_path = '/content/drive/MyDrive/tesis USB /case west reserve university/data_1/outter1.h5'
data_outter_1 = get_h5py_data(file_path)

In [ ]:
data_outter_1.keys()

dict_keys(['161.mat', '162.mat', '163.mat', '164.mat', '262.mat', '263.mat', '264.mat'])

In [20]:
## almacenamiento de la data normal
file_path = "/content/drive/MyDrive/tesis USB /case west reserve university/data_1/normal.h5"
data_normal = get_h5py_data(file_path)

In [ ]:
data_normal.keys()

dict_keys(['100.mat', '97.mat', '98.mat', '99.mat'])

In [ ]:
## almacenamiento de la data ball
file_path = "/content/drive/MyDrive/tesis USB /case west reserve university/data_1/ball.h5"
data_ball = get_h5py_data(file_path)

In [ ]:
data_ball.keys()

dict_keys(['122.mat', '123.mat', '124.mat', '125.mat', '189.mat', '190.mat', '191.mat', '192.mat', '226.mat', '227.mat', '228.mat', '229.mat'])

In [12]:
## calculo el nivel del banco de filtro:
fs=math.ceil(FTF) #frecuencia natural , se usa la del train frequencie
fe=25.6e3 # frecuencia de sampleo
n = math.ceil(math.log(fe / fs) / math.log(2)) +2
level = n

In [ ]:
pywt.wavelist(family=None, kind='discrete')

In [ ]:

 list(data_ball.keys())
# data_outter['190.mat'].keys()
# # data_outter['135.mat']['X135_DE_time']

['122.mat',
 '123.mat',
 '124.mat',
 '125.mat',
 '189.mat',
 '190.mat',
 '191.mat',
 '192.mat',
 '226.mat',
 '227.mat',
 '228.mat',
 '229.mat']

In [ ]:
reshaped_array_healty = data_normal['99.mat']['X099_FE_time'].reshape((data_normal['99.mat']['X099_FE_time'].shape[0],))
reshaped_array_faulty=data_ball['189.mat']['X189_FE_time'].reshape((data_ball['189.mat']['X189_FE_time'].shape[0]))

In [ ]:
coeff_1=cwt_maker(reshaped_array_healty , 'db38', level=level)
coeff_2=cwt_maker(reshaped_array_faulty , 'db38', level=level)

13
13


/usr/local/lib/python3.10/dist-packages/pywt/_multilevel.py:43: UserWarning: Level value of 13 is too high: all coefficients will experience boundary effects.
  warnings.warn(


In [ ]:
# Calculate mean and standard deviation

mean_energy = np.mean(coeff_2)
std_energy = np.std(coeff_2)
# Calculate Z-scores
z_scores = [(energy - mean_energy) / std_energy for energy in coeff_2]

# Normalize the energies_faulty array
normalized_energies_faulty = z_scores

In [ ]:

mean_energy = np.mean(coeff_1)
std_energy = np.std(coeff_1)

# Calculate Z-scores
z_scores = [(energy - mean_energy) / std_energy for energy in coeff_1]

# Normalize the energies_healty array
normalized_energies_healty = z_scores


In [ ]:
df = pd.DataFrame({'Nivel de energia rodamiento sano normalizado ':  normalized_energies_healty, 'Nivel de energia rodamiento bajo falla normalziado': normalized_energies_faulty})
df

,Nivel de energia rodamiento sano normalizado,Nivel de energia rodamiento bajo falla normalziado
0,3.191918,1.815429
1,-0.594733,-0.577481
2,-0.594912,-0.577651
3,-0.595068,-0.577693
4,-0.594879,-0.577989
5,-0.595088,-0.578018
6,-0.532109,-0.543833
7,-0.086996,-0.490392
8,0.011301,-0.223258
9,-0.054528,-0.300397


In [14]:
def get_data(file_path, wavelet, level, exclude_keys=None):
    # Get the data from the H5PY file
    data = get_h5py_data(file_path)

    # Get the order of experiments (sorted keys)
    experiment_order = sorted(data.keys())

    # Create an empty list to store the coefficients
    coefficients_list_DE = []
    coefficients_list_FE = []

    # Process each experiment and calculate coefficients
    for experiment in experiment_order:
        if exclude_keys and experiment in exclude_keys:
            continue  # Skip this experiment

        # Reshape the data for the current experiment
        reshaped_data_DE = data[experiment][list(data[experiment].keys())[1]].reshape(-1) #drive end data
        reshaped_data_FE = data[experiment][list(data[experiment].keys())[2]].reshape(-1) #fan end data

        # Calculate CWT coefficients for the experiment using drive end data
        coefficients_DE = cwt_maker(reshaped_data_DE, wavelet, level)
        coefficientes_normalized_DE = normalization_from_coeffs(coefficients_DE)

        # Calculate CWT coefficients for the experiment using FAN end data
        coefficients_FE = cwt_maker(reshaped_data_FE, wavelet, level)
        coefficientes_normalized_FE = normalization_from_coeffs(coefficients_FE)

        # Append the coefficients to the list
        coefficients_list_DE.append(coefficientes_normalized_DE)
        coefficients_list_FE.append(coefficientes_normalized_FE)

    return coefficients_list_DE, coefficients_list_FE

Calculo de los niveles energeticos normalazidos para falla en el anillo externo

In [17]:
file_path= '/content/drive/MyDrive/tesis USB /case west reserve university/data_1/outter.h5'
normalized_outter_DE,normalized_outter_FE= get_data(file_path, 'db38', level)

/usr/local/lib/python3.10/dist-packages/pywt/_multilevel.py:43: UserWarning: Level value of 13 is too high: all coefficients will experience boundary effects.
  warnings.warn(


13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13


In [ ]:
normalized_outter_FE[0]==df['Nivel de energia rodamiento bajo falla normalziado']

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
Name: Nivel de energia rodamiento bajo falla normalziado, dtype: bool

In [ ]:
normalized_outter_DE[0]==df['Nivel de energia rodamiento bajo falla normalziado']

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
Name: Nivel de energia rodamiento bajo falla normalziado, dtype: bool

Calculo los niveles normalizados asociados a la condicion sana

In [18]:
exclude_keys = {'99.mat', '98.mat'}
file_path=   "/content/drive/MyDrive/tesis USB /case west reserve university/data_1/normal.h5"
normalized_normal_DE,normalized_normal_FE= get_data(file_path, 'db38', level,exclude_keys )

13
13
13
13


In [21]:
### calculo del experimento 98
reshaped_array_healty_FE_98 = data_normal['98.mat']['X098_FE_time'].reshape((data_normal['98.mat']['X098_FE_time'].shape[0],))
reshaped_array_healty_DE_98 = data_normal['98.mat']['X098_DE_time'].reshape((data_normal['98.mat']['X098_DE_time'].shape[0],))
coeff_FE_98=cwt_maker(reshaped_array_healty_FE_98  , 'db38', level=level)
coeff_DE_98=cwt_maker(reshaped_array_healty_DE_98 , 'db38', level=level)

## calculo de los coeficientes normalizados para el  FAN END
mean_energy = np.mean(coeff_FE_98)
std_energy = np.std(coeff_FE_98)
# Calculate Z-scores
z_scores = [(energy - mean_energy) / std_energy for energy in coeff_FE_98]

# Normalize the energies_faulty array
normalized_energies_FE_98= z_scores
del z_scores
## calculo de los coeficientes normalizados para el  DRIVE EN
mean_energy = np.mean(coeff_DE_98)
std_energy = np.std(coeff_DE_98)
# Calculate Z-scores
z_scores = [(energy - mean_energy) / std_energy for energy in coeff_DE_98]

# Normalize the energies_faulty array
normalized_energies_DE_98= z_scores


13
13


In [22]:
### calculo del experimento 99
reshaped_array_healty_FE_99 = data_normal['99.mat']['X099_FE_time'].reshape((data_normal['99.mat']['X099_FE_time'].shape[0],))
reshaped_array_healty_DE_99 = data_normal['99.mat']['X099_DE_time'].reshape((data_normal['99.mat']['X099_DE_time'].shape[0],))
coeff_FE_99=cwt_maker(reshaped_array_healty_FE_99 , 'db38', level=level)
coeff_DE_99=cwt_maker(reshaped_array_healty_DE_99 , 'db38', level=level)

## calculo de los coeficientes normalizados para el  FAN END
mean_energy = np.mean(coeff_FE_99)
std_energy = np.std(coeff_FE_99)
# Calculate Z-scores
z_scores = [(energy - mean_energy) / std_energy for energy in coeff_FE_99]

# Normalize the energies_faulty array
normalized_energies_FE_99= z_scores
del z_scores
## calculo de los coeficientes normalizados para el  DRIVE EN
mean_energy = np.mean(coeff_DE_99)
std_energy = np.std(coeff_DE_99)
# Calculate Z-scores
z_scores = [(energy - mean_energy) / std_energy for energy in coeff_DE_99]

# Normalize the energies_faulty array
normalized_energies_DE_99= z_scores

13
13


In [ ]:
del normalized_normal_FE,normalized_normal_DE

In [23]:
#añado los experimentos a la lista general
normalized_normal_FE.extend([normalized_energies_FE_98,normalized_energies_FE_99])
normalized_normal_DE.extend([normalized_energies_DE_98,normalized_energies_DE_99])

In [ ]:
normalized_normal_DE[3]==df['Nivel de energia rodamiento sano normalizado ']

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
Name: Nivel de energia rodamiento sano normalizado , dtype: bool

In [ ]:
normalized_normal_FE[3]==df['Nivel de energia rodamiento sano normalizado ']

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
Name: Nivel de energia rodamiento sano normalizado , dtype: bool

Calculo de los niveles energeticos para falla en el anillo externo pero con el acelerometro a las 6

In [24]:
file_path=  "/content/drive/MyDrive/tesis USB /case west reserve university/data_1/outter1.h5"
normalized_outter_1_DE,normalized_outter_1_FE= get_data(file_path, 'db38', level)

13
13
13
13
13
13
13
13
13
13
13
13
13
13


In [ ]:
normalized_outter_1_FE[0]==df['Nivel de energia rodamiento bajo falla normalziado']

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
Name: Nivel de energia rodamiento bajo falla normalziado, dtype: bool

In [ ]:
normalized_outter_1_DE[0]==df['Nivel de energia rodamiento bajo falla normalziado']

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
Name: Nivel de energia rodamiento bajo falla normalziado, dtype: bool

Calculo de los niveles energeticos para falla en el anillo interno

In [25]:
file_path=   "/content/drive/MyDrive/tesis USB /case west reserve university/data_1/inner.h5"
normalized_inner_DE,normalized_inner_FE= get_data(file_path, 'db38', level)

13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13


In [ ]:
normalized_inner_DE[0]==df['Nivel de energia rodamiento bajo falla normalziado']

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
Name: Nivel de energia rodamiento bajo falla normalziado, dtype: bool

In [ ]:
normalized_inner_FE[0]==df['Nivel de energia rodamiento bajo falla normalziado']

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
Name: Nivel de energia rodamiento bajo falla normalziado, dtype: bool

Calculo de los niveles energeticos para falla en el elemento rodante

In [26]:
file_path="/content/drive/MyDrive/tesis USB /case west reserve university/data_1/ball.h5"
normalized_ball_DE,normalized_ball_FE= get_data(file_path, 'db38', level)

13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13


In [ ]:
normalized_ball_DE[4]==df['Nivel de energia rodamiento bajo falla normalziado']

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
Name: Nivel de energia rodamiento bajo falla normalziado, dtype: bool

In [ ]:
normalized_ball_FE[4]==df['Nivel de energia rodamiento bajo falla normalziado']

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
Name: Nivel de energia rodamiento bajo falla normalziado, dtype: bool

In [ ]:
# Find the maximum length (number of columns)
max_length = max(len(sublist) for sublist in normalized_ball_DE)

# Create an empty array filled with a placeholder (e.g., np.nan)
array_1 = np.full((len(normalized_ball_DE), max_length), np.nan)

# Fill the array with the elements from your lists
for i, sublist in enumerate(normalized_ball_DE):
    array_1[i, :len(sublist)] = sublist

    # Find the maximum length (number of columns)
max_length = max(len(sublist) for sublist in normalized_ball_FE)

# Create an empty array filled with a placeholder (e.g., np.nan)
array_2 = np.full((len(normalized_ball_FE), max_length), np.nan)

# Fill the array with the elements from your lists
for i, sublist in enumerate(normalized_ball_FE):
    array_2[i, :len(sublist)] = sublist

In [ ]:


# Assuming you have array_2 and array_1
array_3 = np.concatenate((array_2, array_1), axis=0)
array_3.shape

(24, 14)

### creacion y concatenacion de los arrays asociados a la condicion normal

In [27]:
### creacion y concatenacion de los arrays asociados a la condicion normal
# Assuming normalized_normal_FE and normalized_normal_DE are lists
array_normal_FE = np.array(normalized_normal_FE)
array_normal_DE = np.array(normalized_normal_DE)

# Concatenate the arrays along axis 0 (stack them vertically)
array_normal = np.concatenate([array_normal_FE, array_normal_DE], axis=0)
array_normal.shape

(8, 14)

In [ ]:
are_equal = np.array_equal(array_3,array_normal)

if are_equal:
    print("The arrays are equal.")
else:
    print("The arrays are not equal.")

The arrays are equal.


### creacion y concatenacion de los arrays asociados a la condicion de falla externa

In [28]:
### creacion y concatenacion de los arrays asociados a la condicion de falla externa
array_outter_FE = np.array(normalized_outter_FE)
array_outter_DE = np.array(normalized_outter_DE)

# Concatenate the arrays along axis 0 (stack them vertically)
array_outter = np.concatenate([array_outter_FE, array_outter_DE ], axis=0)
array_outter.shape

(34, 14)

In [ ]:
are_equal = np.array_equal(array_3, array_outter )

if are_equal:
    print("The arrays are equal.")
else:
    print("The arrays are not equal.")

The arrays are equal.


### creacion y concatenacion de los arrays asociados a la condicion de falla externo pero con el acelerometro en otra disposicion

In [29]:
### creacion y concatenacion de los arrays asociados a la condicion de falla externa
array_outter_1_FE = np.array(normalized_outter_1_FE)
array_outter_1_DE = np.array(normalized_outter_1_DE)

# Concatenate the arrays along axis 0 (stack them vertically)
array_outter_1 = np.concatenate([array_outter_1_FE, array_outter_1_DE ], axis=0)
array_outter_1.shape

(14, 14)

In [ ]:
are_equal = np.array_equal(array_3, array_outter_1)

if are_equal:
    print("The arrays are equal.")
else:
    print("The arrays are not equal.")

The arrays are equal.


### creacion y concatenacion de los arrays asociados a la condicion de falla interna

In [30]:
### creacion y concatenacion de los arrays asociados a la condicion de falla interna
array_inner_FE = np.array(normalized_inner_FE)
array_inner_DE = np.array(normalized_inner_DE)

# Concatenate the arrays along axis 0 (stack them vertically)
array_inner = np.concatenate([array_inner_FE, array_inner_DE ], axis=0)
array_inner.shape

(24, 14)

In [ ]:
len(array_inner)

24

In [ ]:
are_equal = np.array_equal(array_3, array_inner)

if are_equal:
    print("The arrays are equal.")
else:
    print("The arrays are not equal.")

The arrays are equal.


### creacion y concatenacion de los arrays asociados a la condicion de falla en el elemento rodante

In [31]:
### creacion y concatenacion de los arrays asociados a la condicion de falla interna
array_ball_FE = np.array(normalized_ball_FE)
array_ball_DE = np.array(normalized_ball_DE)

# Concatenate the arrays along axis 0 (stack them vertically)
array_ball = np.concatenate([array_ball_FE, array_ball_DE ], axis=0)
array_ball.shape

(24, 14)

In [ ]:
are_equal = np.array_equal(array_3, array_ball)

if are_equal:
    print("The arrays are equal.")
else:
    print("The arrays are not equal.")

The arrays are equal.


### creacion de los targets

In [32]:
def create_mask_array(variable, variable_name, substring):
    # Create an array to store the mask values
    if substring in variable_name:
        mask_array = np.zeros((len(variable), 1))
    else:
        mask_array = np.ones((len(variable), 1))
    return mask_array

In [33]:
normal_targets=create_mask_array(array_normal, "array_normal", "normal")

In [ ]:
normal_targets.shape

(8, 1)

In [34]:
outter_targets=create_mask_array(array_outter, "array_outter", "normal")

In [ ]:
outter_targets

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [35]:
outter_targets_1=create_mask_array(array_outter_1, "array_outter_1", "normal")

In [ ]:
outter_targets_1

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [36]:
inner_targets=create_mask_array(array_inner, "array_inner", "normal")

In [ ]:
inner_targets.shape

(24, 1)

In [37]:
ball_targets=create_mask_array(array_ball, "array_ball", "normal")

In [ ]:
ball_targets

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]])

### concatenacion de los targets

In [38]:
y_targets = np.concatenate((normal_targets, outter_targets, outter_targets_1, inner_targets,ball_targets))

In [39]:
y_targets.shape

(104, 1)

### concatenacion de los features

In [45]:
x_features=np.concatenate((array_normal, array_outter, array_outter_1, array_inner,array_ball))

In [46]:
x_features.shape

(104, 14)

#### Almaceno la nueva data en google drive





In [49]:
#creo el h5py de las features
data=x_features
path_features="/content/drive/MyDrive/tesis USB /case west reserve university/data_1/features"
hdf5_file_path = path_features
# with h5py.File(hdf5_file_path, 'w') as hf:
#     hf.create_dataset('features', data=data)
with h5py.File(hdf5_file_path, 'r') as hf:
    x_features_1 = hf['features'][:]


In [51]:
are_equal = np.array_equal(x_features, x_features_1)

if are_equal:
    print("The arrays are equal.")
else:
    print("The arrays are not equal.")

The arrays are equal.


In [52]:
#creo el h5py de las features
data=y_targets
path_features="/content/drive/MyDrive/tesis USB /case west reserve university/data_1/targets"
hdf5_file_path = path_features
# with h5py.File(hdf5_file_path, 'w') as hf:
#     hf.create_dataset('targets', data=data)
with h5py.File(hdf5_file_path, 'r') as hf:
    y_targets_1 = hf['targets'][:]


In [54]:
are_equal = np.array_equal(y_targets, y_targets_1)

if are_equal:
    print("The arrays are equal.")
else:
    print("The arrays are not equal.")

The arrays are equal.
